In [ ]:
#!/usr/bin/env python

import pycarwings2
import time
from configparser import ConfigParser
import logging
import sys
from tplink_smartplug import SmartPlug

plug = SmartPlug('192.168.1.116')

print("Enchufe 1:")
print('Name:      %s' % plug.name)
print('Model:     %s' % plug.model)
print('Mac:       %s' % plug.mac)
print('Time:      %s' % plug.time)

print('Is on:     %s' % plug.is_on)
print('Nightmode: %s' % (not plug.led))
print('RSSI:      %s' % plug.rssi)

plug2 = SmartPlug('192.168.1.118')

print("Enchufe 2:")
print('Name:      %s' % plug2.name)
print('Model:     %s' % plug2.model)
print('Mac:       %s' % plug2.mac)
print('Time:      %s' % plug2.time)

print('Is on:     %s' % plug2.is_on)
print('Nightmode: %s' % (not plug2.led))
print('RSSI:      %s' % plug2.rssi)

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger("pycarwings2").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

parser = ConfigParser()
candidates = ['config.ini', 'my_config.ini']
found = parser.read(candidates)

username = parser.get('get-leaf-info', 'username')
password = parser.get('get-leaf-info', 'password')
username2 = parser.get('get-leaf-info', 'username2')
password2 = parser.get('get-leaf-info', 'password2')
region = parser.get('get-leaf-info', 'region')
enchufe1 = parser.get('get-leaf-info', 'enchufe1')
enchufe2 = parser.get('get-leaf-info', 'enchufe2')

todaypriority = int(parser.get('get-leaf-info', 'todaypriority'))
leaf1min = int(parser.get('get-leaf-info', 'leaf1min'))
leaf2min = int(parser.get('get-leaf-info', 'leaf2min'))
leaf1tgt = int(parser.get('get-leaf-info', 'leaf1tgt'))
leaf2tgt = int(parser.get('get-leaf-info', 'leaf2tgt'))
leaf1max = int(parser.get('get-leaf-info', 'leaf1max'))
leaf2max = int(parser.get('get-leaf-info', 'leaf2max'))


sleepsecs = 10     # Time to wait before polling Nissan servers for update
sleepsecs2 = 10     # Time to wait before polling Nissan servers for update

def wait_update_battery_status(leaf,key,wait_time=1,retries=3,wait_time_retries=1):
    status = leaf.get_status_from_update(key)
    # Currently the nissan servers eventually return status 200 from get_status_from_update(), previously
    # they did not, and it was necessary to check the date returned within get_latest_battery_status().
    if status is None:
        print("Retrying in {0} seconds".format(wait_time))
        time.sleep(wait_time)
        status = leaf.get_status_from_update(key)
        
    if status is None:
        r = retries - 1
    
    while (status is None) and (r > 0):
        print("Retrying in {0} seconds".format(wait_time_retries))
        time.sleep(wait_time_retries)
        status = leaf.get_status_from_update(key)
        r = r - 1
    return status


def print_info(info):
    print("  date %s" % info.answer["BatteryStatusRecords"]["OperationDateAndTime"])
    #print("  date %s" % info.answer["BatteryStatusRecords"]["NotificationDateAndTime"])
    #print("  battery_capacity2 %s" % info.answer["BatteryStatusRecords"]["BatteryStatus"]["BatteryCapacity"])
    print("  battery_capacity %s" % info.battery_capacity)
    print("  charging_status %s" % info.charging_status)
    #print("  battery_capacity %s" % info.battery_capacity)
    #print("  battery_remaining_amount %s" % info.battery_remaining_amount)
    #print("  charging_status %s" % info.charging_status)
    #print("  is_charging %s" % info.is_charging)
    #print("  is_quick_charging %s" % info.is_quick_charging)
    print("  plugin_state %s" % info.plugin_state)
    #print("  is_connected %s" % info.is_connected)
    #print("  is_connected_to_quick_charger %s" % info.is_connected_to_quick_charger)
    #print("  time_to_full_trickle %s" % info.time_to_full_trickle)
    print("  time_to_full_l2 %s" % info.time_to_full_l2)
    #print("  time_to_full_l2_6kw %s" % info.time_to_full_l2_6kw)
    print("  battery_percent %s" % info.battery_percent)
    #print("  state_of_charge %s" % info.state_of_charge)

def turn_1_on():
    global plug
    global plug2
    global leaf
    plug2.turn_off()
    time.sleep(5)
    plug.turn_on()
    time.sleep(5)
    #resp=leaf.start_charging()
    resp="Ok"
    print("Charging car 1:",resp)
    
def turn_off():
    global plug
    global plug2
    plug.turn_off()
    time.sleep(5)
    plug2.turn_off()
    
def turn_2_on():
    global plug
    global plug2
    global leaf2
    plug.turn_off()
    time.sleep(5)
    plug2.turn_on()
    time.sleep(5)
    #resp=leaf2.start_charging()
    resp="Ok"
    print("Charging car 2:",resp)    
    
# Main program

#logging.debug("login = %s, password = %s, region = %s" % (username, password, region))
#logging.debug("login2 = %s, password2 = %s, region = %s" % (username2, password2, region))

print("Prepare Session 1")
s = pycarwings2.Session(username, password, region)
print("Login...1")
leaf = s.get_leaf()

print("Prepare Session 2")
s2 = pycarwings2.Session(username2, password2, region)
print("Login...2")
leaf2 = s2.get_leaf()

while(True):
    # Give the nissan servers a bit of a delay so that we don't get stale data'
    time.sleep(1)

    print("********** First Car Last Status ************")
    print("get_latest_battery_status from servers")
    leaf_info = leaf.get_latest_battery_status()
    #start_date = leaf_info.answer["BatteryStatusRecords"]["OperationDateAndTime"]
    #print("start_date=", start_date)
    print_info(leaf_info)
    print("request an update from the car itself")
    key = leaf.request_update()

    print("********** Second Car Last Status ************")
    print("get_latest_battery_status from servers")
    leaf_info2 = leaf2.get_latest_battery_status()
    #start_date2 = leaf_info2.answer["BatteryStatusRecords"]["OperationDateAndTime"]
    #print("start_date2=", start_date2)
    print_info(leaf_info2)
    print("request an update from the car itself")
    key2 = leaf2.request_update()

    # Give the nissan servers a bit of a delay so that we don't get stale data
    time.sleep(1)
    print("***** Waiting for status update *****")
    print("... First car")
    update_status = wait_update_battery_status(leaf,key,sleepsecs,3,5)
    print("... Second car")
    update_status2 = wait_update_battery_status(leaf2,key2,sleepsecs2,3,5)

    print("********** First Car Current Status************")
    if (update_status is not None):
        print("OK: >>>>",update_status.answer['status'])
        latest_leaf_info = leaf.get_latest_battery_status()
        #latest_date = latest_leaf_info.answer["BatteryStatusRecords"]["OperationDateAndTime"]
        #print("latest_date=", latest_date)
        print_info(latest_leaf_info)
    else:
        print("ERROR: >>>> status could not be retrieved")    

    print("********** Second Car Current Status************")
    if (update_status2 is not None):
        print("OK: >>>>",update_status2.answer['status'])
        latest_leaf_info2 = leaf2.get_latest_battery_status()
        #latest_date2 = latest_leaf_info2.answer["BatteryStatusRecords"]["OperationDateAndTime"]
        #print("latest_date2=", latest_date2)
        print_info(latest_leaf_info2)
    else:
        print("ERROR: >>>> status could not be retrieved")

    bat1 = latest_leaf_info.battery_percent
    bat2 = latest_leaf_info2.battery_percent
    
    '''
    todaypriority = 1
    leaf1min = 80
    leaf2min = 20
    leaf1tgt = 90
    leaf2tgt = 50
    leaf1max = 100
    leaf2max = 85
    '''
    if (todaypriority == 1):
        if (bat1 < leaf1min):
            print("Caso 1")
            turn_1_on()
        else:
            if (bat2 < leaf2min):
                print("Caso 2")
                turn_2_on()
            else:
                if (bat1 < leaf1tgt):
                    print("Caso 3")
                    turn_1_on()
                else:
                    if (bat2 < leaf2tgt):
                        print("Caso 4")
                        turn_2_on();
                    else:
                        if (bat1 < leaf1max):
                            print("Caso 5")
                            turn_1_on()
                        else:
                            if (bat2 < leaf2max):
                                print("Caso 6")
                                turn_2_on()
                            else:
                                print("Caso 7")
                                turn_off()   
        
    print('Plug 1 Is on:     %s' % plug.is_on)
    print('Plug 2 Is on:     %s' % plug2.is_on)
    print("Durmiendo 1 minuto")
    time.sleep(60)
        
                

********** First Car Last Status ************
get_latest_battery_status from servers
dict_keys(['VoltLabel', 'BatteryStatusRecords', 'status'])
  date 08-ago-2020 02:27
  battery_capacity 240
  charging_status NORMAL_CHARGING
  plugin_state CONNECTED
  time_to_full_l2 9:00:00
  battery_percent 53.0
request an update from the car itself


----- This is another API

In [ ]:
from leafpy import Leaf
leaf = Leaf('txinto', 'guishi01',region_code='NE')

In [ ]:
batinfo = leaf.BatteryStatusRecordsRequest()

In [ ]:
print(batinfo)

In [ ]:
leaf.BatteryStatusRecordsRequest()

In [ ]:
response = leaf.BatteryStatusCheckRequest()

In [ ]:
leaf.BatteryStatusCheckResultRequest(resultKey=response['resultKey'])

In [ ]:
from leafpy import Leaf
leaf = Leaf('monicalcobre@gmail.com', 'cosita01',region_code='NE')